In [1]:
import httpx
from api.src.schemas.authorities.authority import Uri, Variant, AdminMetadata, Element
from typing import Optional
from pydantic import BaseModel
from pysolr import Solr
from pyfuseki import FusekiUpdate

In [22]:
au_update = FusekiUpdate('http://localhost:3030', 'authorities')

d = """DELETE { graph ?g { ?s ?p ?o } } 
        WHERE {
        graph ?g {?s ?p ?o.}
        }"""

response = au_update.run_sparql(d)
response.convert()

solr = Solr('http://localhost:8983/solr/authorities/', timeout=10)
solr.delete(q="*:*",  commit=True)

'<?xml version="1.0" encoding="UTF-8"?>\n<response>\n\n<lst name="responseHeader">\n  <int name="status">0</int>\n  <int name="QTime">18</int>\n</lst>\n</response>\n'

In [3]:
authority = 'http://id.loc.gov/authorities/subjects/sh85050603'
url = f'http://localhost:8000/authorities/import/loc/subject?uri={authority}'
r = httpx.get(url, timeout=20)
r.json()

{'type': 'Topic',
 'adminMetadata': {'assigner': 'http://id.loc.gov/vocabulary/organizations/dlc',
  'descriptionModifier': 'http://id.loc.gov/vocabulary/organizations/brmninpa',
  'creationDate': '2023-05-19',
  'descriptionLanguage': 'http://id.loc.gov/vocabulary/languages/por',
  'generationProcess': 'BiblioKeia v.1',
  'generationDate': '2023-05-19T15:01:38',
  'identifiedBy': [{'type': 'Lccn',
    'assigner': 'http://id.loc.gov/vocabulary/organizations/dlc',
    'value': 'sh85050603'}],
  'status': {'value': 'mstatus:new', 'label': 'novo'}},
 'elementList': [{'type': 'TopicElement',
   'elementValue': {'value': 'Gestão florestal', 'lang': 'pt'}}],
 'note': None,
 'hasReciprocalAuthority': None,
 'hasReciprocalExternalAuthority': [{'value': 'http://id.loc.gov/authorities/subjects/sh2001005656',
   'label': {'value': 'Forest plants--Control', 'lang': 'en'},
   'base': 'loc'}],
 'hasBroaderAuthority': None,
 'hasBroaderExternalAuthority': [{'value': 'http://id.loc.gov/authorities/sub

In [20]:
headers = {
    'accept': 'application/json',
    'Content-Type': 'application/json',
}
response = httpx.post('http://localhost:8000/authorities/subject/', headers=headers, json=r.json())
response.json()

{'id': 'bkau-2',
 'jena': 'Update succeeded',
 'solr': '{\n  "responseHeader":{\n    "status":0,\n    "QTime":165}}\n'}

In [8]:
class Subject(BaseModel):
    type: str 
    adminMetadata: AdminMetadata 
    elementList: list[Element]
    note: Optional[str]
    hasReciprocalAuthority: Optional[list[Uri]]
    hasReciprocalExternalAuthority: Optional[list[Uri]]
    hasBroaderAuthority: Optional[list[Uri]]
    hasBroaderExternalAuthority: Optional[list[Uri]]
    hasNarrowerAuthority: Optional[list[Uri]]
    hasNarrowerExternalAuthority: Optional[list[Uri]]
    hasCloseExternalAuthority: Optional[list[Uri]]
    hasExactExternalAuthority: Optional[list[Uri]]
    hasVariant: Optional[list[Variant]]
    subjectOf: Optional[list[Uri]]
    isMemberOfMADSCollection: str

In [19]:
authority = 'http://id.loc.gov/authorities/subjects/sh93001294'
url = f'http://localhost:8000/authorities/import/loc/subject?uri={authority}'
r = httpx.get(url, timeout=20)
r.json()

{'type': 'Topic',
 'adminMetadata': {'assigner': 'http://id.loc.gov/vocabulary/organizations/dlc',
  'descriptionModifier': 'http://id.loc.gov/vocabulary/organizations/brmninpa',
  'creationDate': '2023-05-19',
  'descriptionLanguage': 'http://id.loc.gov/vocabulary/languages/por',
  'generationProcess': 'BiblioKeia v.1',
  'generationDate': '2023-05-19T16:19:42',
  'identifiedBy': [{'type': 'Lccn',
    'assigner': 'http://id.loc.gov/vocabulary/organizations/dlc',
    'value': 'sh93001294'}],
  'status': {'value': 'mstatus:new', 'label': 'novo'}},
 'elementList': [{'type': 'TopicElement',
   'elementValue': {'value': 'Gestão do ecossistema', 'lang': 'pt'}}],
 'note': None,
 'hasReciprocalAuthority': None,
 'hasReciprocalExternalAuthority': [{'value': 'http://id.loc.gov/authorities/subjects/sh87005570',
   'label': {'value': 'Biodiversity conservation', 'lang': 'en'},
   'base': 'loc'}],
 'hasBroaderAuthority': None,
 'hasBroaderExternalAuthority': [{'value': 'http://id.loc.gov/authoriti

In [17]:
def MakeLabel(elementList):
    labels = [i.elementValue.value for i in elementList]
    label = ", ".join(labels)
    
    return label

def MakeDocSubject(request, id):
    doc = {
            'id': id,
            'type': request.type,
            "creationDate": request.adminMetadata.creationDate.strftime('%Y-%m-%d'), 
            "label": f'{MakeLabel(request.elementList)}' ,
            "isMemberOfMADSCollection": request.isMemberOfMADSCollection
        }
    if request.hasVariant:
        variants = list()
        for i in request.hasVariant:
            label = [j.elementValue.value for j in i.elementList]
            label = "--".join(label)
            variants.append(label)
        doc['variant'] = variants
    for k, v in request:
        if v and k not in ['type', 'adminMetadata', 'elementList', 'hasVariant', 'isMemberOfMADSCollection']:
                uris = list()
                for i in v:
                        uri = {
                                'id': f"{id}/{k}#{i.value.split('/')[-1]}",
                                'uri': i.value, 
                                'label': i.label.value, 
                                'lang': i.label.lang,
                                'base': i.base
                                }
                        uris.append(uri)
                doc[f'{k}'] = uris
    return doc

In [11]:
doc = {
        'id': id,
        'type': request.type,
        "creationDate": request.adminMetadata.creationDate.strftime('%Y-%m-%d'), 
        "label": f'{MakeLabel(request.elementList)}' ,
        "isMemberOfMADSCollection": request.isMemberOfMADSCollection
        }
doc

{'id': 'bkau-1',
 'type': 'Topic',
 'creationDate': '2023-05-18',
 'label': 'Matemática',
 'isMemberOfMADSCollection': 'http://bibliokeia.com/authorities/Topic/'}

In [16]:
request.hasBroaderExternalAuthority

[Uri(value='http://id.loc.gov/authorities/subjects/sh85118553', label=Label(value='Science', lang='en'), base='loc')]

In [18]:
doc = MakeDocSubject(request, id)
doc

{'id': 'bkau-1',
 'type': 'Topic',
 'creationDate': '2023-05-18',
 'label': 'Matemática',
 'isMemberOfMADSCollection': 'http://bibliokeia.com/authorities/Topic/',
 'variant': ['Matemática'],
 'hasBroaderExternalAuthority': [{'id': 'bkau-1/hasBroaderExternalAuthority#sh85118553',
   'uri': 'http://id.loc.gov/authorities/subjects/sh85118553',
   'label': 'Science',
   'lang': 'en',
   'base': 'loc'}],
 'hasNarrowerExternalAuthority': [{'id': 'bkau-1/hasNarrowerExternalAuthority#sh85082129',
   'uri': 'http://id.loc.gov/authorities/subjects/sh85082129',
   'label': 'Mathematical physics',
   'lang': 'en',
   'base': 'loc'},
  {'id': 'bkau-1/hasNarrowerExternalAuthority#sh85148106',
   'uri': 'http://id.loc.gov/authorities/subjects/sh85148106',
   'label': 'Word problems (Mathematics)',
   'lang': 'en',
   'base': 'loc'},
  {'id': 'bkau-1/hasNarrowerExternalAuthority#sh2021005313',
   'uri': 'http://id.loc.gov/authorities/subjects/sh2021005313',
   'label': 'Classification--Books--Mathemat

In [13]:
solr = Solr('http://localhost:8983/solr/authorities/', timeout=10)

doc = {
 "id":"bkau-2",
 "hasNarrowerExternalAuthority":{ "add": ["TESTE 2" ]}
}

responseSolr = solr.add([doc], commit=True)

In [21]:
solr.delete(q=f"id:bkau-2/hasNarrowerAuthority#TESTE",  commit=True)

'<?xml version="1.0" encoding="UTF-8"?>\n<response>\n\n<lst name="responseHeader">\n  <int name="status">0</int>\n  <int name="QTime">66</int>\n</lst>\n</response>\n'

{
  "authority": "https://bibliokeia.com/authorities/Topic/bkau-1",
  "uri": "https://bibliokeia.com/authorities/Topic/TESTE",
  "value": "CONTROLE FLORESTAL",
  "lang": "pt",
  "type": "hasReciprocalExternalAuthority"
}